In [ ]:
import pandas as pd

df = pd.read_csv('../data/task2.csv')

In [ ]:
import os
import json

draft_ids = list(df['Original_id'])

# input data
countries = list(df['Country'])
drafts = []
votes = list(df['Voting'])

path = '../data/task2'
for i in draft_ids:
    folder_path = os.path.join(path, str(i))
    files = os.listdir(folder_path)
    json_file = [file for file in files if file.endswith('EN.json')][0]
    with open(os.path.join(folder_path, json_file)) as f:
        draft = json.load(f)
    drafts.append(draft['Content'])

In [ ]:
# If use Together API
from together import Together

your_model_name = 'xxxxxxxxxxxxxxxxxxxxxxxx' 
your_api_key = 'xxxxxxxxxxxxxxxxxxxxxxxx'
client = Together(api_key=your_api_key)

In [ ]:
import random
from tqdm import tqdm

pred = []
invalid_responses = []
for i, (draft, country) in tqdm(enumerate(zip(drafts, countries))):
    system_prompt = "You are an experienced diplomat participating in United Nations Security Council sessions. Your task is to read draft resolutions and vote accordingly. You must strictly respond with 'Y', 'N', or 'A' without any additional explanation."
    user_prompt = f"""The following is a United Nations Security Council draft resolution. Assume you are a diplomat from {country} and must cast your vote.

    Please read the draft and vote using one of the following options:
    - 'Y' for Yes (in favor)
    - 'N' for No (against)
    - 'A' for Abstain

    Draft Resolution: {draft}
    Answer:
    """
    response = client.chat.completions.create(
        model=your_model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1, 
        temperature=0.0
    )
    result = response.choices[0].message.content.strip()
    valid_votes = ['Y', 'N', 'A']
    if result not in valid_votes:
        print(f"Invalid response: {result}")
        result = random.choice(valid_votes)
        invalid_responses.append(i)
    pred.append(result)

In [ ]:
# calculate metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder, label_binarize
from imblearn.metrics import geometric_mean_score
import numpy as np

def calculate_metrics(pred, labels):
    label_encoder = LabelEncoder()
    all_classes = list(set(labels) | set(pred))  
    label_encoder.fit(all_classes)

    labels = label_encoder.transform(labels) 
    pred = label_encoder.transform(pred)  

    acc = accuracy_score(labels, pred)
    
    num_classes = len(label_encoder.classes_)
    true_labels_bin = label_binarize(labels, classes=list(range(num_classes)))
    pred_bin = label_binarize(pred, classes=list(range(num_classes)))  

    auc = roc_auc_score(true_labels_bin, pred_bin, multi_class='ovr', average='macro')
    pr_auc = average_precision_score(true_labels_bin, pred_bin, average='macro')

    balanced_acc = balanced_accuracy_score(labels, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average='macro')

    mcc = matthews_corrcoef(labels, pred)
    g_mean = geometric_mean_score(labels, pred, average='macro')

    print(f'Accuracy: {acc}')
    print(f'AUC: {auc}')
    print(f'Balanced Accuracy: {balanced_acc}')
    print(f'Precision: {prec}')
    print(f'Recall: {rec}')
    print(f'F1: {f1}')
    print(f'PR AUC: {pr_auc}')
    print(f'MCC: {mcc}')
    print(f'G-Mean: {g_mean}')

    print('Accuracy AUC Balanced_Acc Precision Recall F1 PR_AUC MCC G-Mean')
    print(f'{acc:.4f} {auc:.4f} {balanced_acc:.4f} {prec:.4f} {rec:.4f} {f1:.4f} {pr_auc:.4f} {mcc:.4f} {g_mean:.4f}')


In [ ]:
calculate_metrics(pred, votes)